In [1]:
import numpy as np

In [29]:
with open('movie-xids.npy', 'rb') as f:
    Xids = np.load(f, allow_pickle = True).astype("int32")
    
with open('movie-xmask.npy', 'rb') as f:
    Xmask = np.load(f, allow_pickle = True).astype("int32")
    
with open('movie-labels.npy', 'rb') as f:
    labels = np.load(f, allow_pickle = True)

In [30]:
Xids.shape

(156060, 512)

In [31]:
labels.shape

(156060, 5)

In [32]:
labels

array([[0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.]])

In [33]:
import tensorflow as tf

In [34]:
dataset = tf.data.Dataset.from_tensor_slices((Xids, Xmask, labels))

In [35]:
dataset.take(1)

<TakeDataset shapes: ((512,), (512,), (5,)), types: (tf.int32, tf.int32, tf.float64)>

In [36]:
def map_function(input_ids, masks, labels):
  return {'input_ids': input_ids,
          'attention_mask': masks}, labels

In [37]:
dataset = dataset.map(map_function)

In [38]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (512,), attention_mask: (512,)}, (5,)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [39]:
batch_size = 16

In [40]:
dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder = True)

In [41]:
dataset.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>

In [42]:
split = 0.9

In [43]:
size = int(Xids.shape[0] / batch_size * split)
size

8778

In [44]:
train_ds = dataset.take(size)
val_ds = dataset.skip(size)

In [45]:
tf.data.experimental.save(train_ds, 'train')
tf.data.experimental.save(val_ds, 'val')

In [46]:
train_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [48]:
val_ds.element_spec

({'input_ids': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(16, 512), dtype=tf.int32, name=None)},
 TensorSpec(shape=(16, 5), dtype=tf.float64, name=None))

In [49]:
val_ds.element_spec == train_ds.element_spec

True

In [50]:
ds = tf.data.experimental.load('train', element_spec = train_ds.element_spec)

In [51]:
ds.take(1)

<TakeDataset shapes: ({input_ids: (16, 512), attention_mask: (16, 512)}, (16, 5)), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float64)>